In [1]:
import kagglehub
import pandas as pd
import os

local_path = kagglehub.dataset_download("erlichsefi/israeli-supermarkets-2024")

selected_supermarkets = [
    "shufersal", "rami_levy", "victory", "tiv_taam", "yohananof", "osher_ad", "mega"
]

save_dir='data'
product_dfs = []
for market in selected_supermarkets:
    filename = f"price_file_{market}.csv"
    file_path = os.path.join(local_path, filename)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        df["supermarket"] = market
        product_dfs.append(df)

        df.to_csv(os.path.join(save_dir, filename), index=False)
        print(f"✅ Loaded & Saved: {filename} ({len(df)} rows)")
    else:
        print(f"❌ Missing: {filename}")


if product_dfs:
    all_products = pd.concat(product_dfs, ignore_index=True)
    combined_path = os.path.join(save_dir, "combined_supermarket_products.csv")
    all_products.to_csv(combined_path, index=False)
    print(f"\n📦 Total products: {len(all_products)}")
    print(f"✅ Combined data saved to: {combined_path}")
    print(all_products.head())
else:
    print("❌ No product data loaded.")


✅ Loaded & Saved: price_file_shufersal.csv (854248 rows)


C:\Users\Nagham Omar\AppData\Local\Temp\ipykernel_30360\1759887513.py:17: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


✅ Loaded & Saved: price_file_rami_levy.csv (346443 rows)
✅ Loaded & Saved: price_file_victory.csv (4043 rows)
✅ Loaded & Saved: price_file_tiv_taam.csv (6682 rows)
✅ Loaded & Saved: price_file_yohananof.csv (24514 rows)
✅ Loaded & Saved: price_file_osher_ad.csv (3792 rows)
✅ Loaded & Saved: price_file_mega.csv (155509 rows)

📦 Total products: 1395231
✅ Combined data saved to: data\combined_supermarket_products.csv
               found_folder                                file_name  \
0  app_data/dumps/Shufersal  Price7290027600007-131-202506180200.xml   
1                       NaN                                      NaN   
2                       NaN                                      NaN   
3                       NaN                                      NaN   
4                       NaN                                      NaN   

        chainid  subchainid  storeid  bikoretno   priceupdatedate  \
0  7.290028e+12         7.0    131.0        9.0  2025-06-17 05:30   
1          

In [2]:

import kagglehub
import pandas as pd
import os
data_path = kagglehub.dataset_download("erlichsefi/israeli-supermarkets-2024")

supermarkets = [
    "shufersal", "rami_levy", "victory", "tiv_taam", "yohananof", "osher_ad", "mega"
]

product_dfs = []
promo_dfs = []

for market in supermarkets:
    prod_file = f"price_full_file_{market}.csv"
    promo_file = f"promo_full_file_{market}.csv"
    prod_path = os.path.join(data_path, prod_file)
    promo_path = os.path.join(data_path, promo_file)

    if os.path.exists(prod_path):
        df_prod = pd.read_csv(prod_path, low_memory=False)
        df_prod["supermarket"] = market
        product_dfs.append(df_prod)
        print(f"✅ Loaded product file for {market}")
    else:
        print(f"❌ Product file not found for {market}")

    if os.path.exists(promo_path):
        df_promo = pd.read_csv(promo_path, low_memory=False)
        df_promo["supermarket"] = market
        promo_dfs.append(df_promo)
        print(f"✅ Loaded promo file for {market}")
    else:
        print(f"❌ Promo file not found for {market}")

if not product_dfs:
    raise ValueError("No product data loaded.")

all_products = pd.concat(product_dfs, ignore_index=True)

if promo_dfs:
    all_promos = pd.concat(promo_dfs, ignore_index=True)
else:
    all_promos = pd.DataFrame()

# Ensure 'priceupdatedate' is parsed
if 'priceupdatedate' in all_products.columns:
    all_products['priceupdatedate'] = pd.to_datetime(all_products['priceupdatedate'], errors='coerce')
    # Keep only the most recent for each (itemcode, supermarket)
    all_products = all_products.sort_values('priceupdatedate', ascending=False)
    products_latest = all_products.drop_duplicates(subset=['itemcode', 'supermarket'], keep='first')
else:
    products_latest = all_products.drop_duplicates(subset=['itemcode', 'supermarket'], keep='first')

# Merge promo info (add promo columns to product info, do not drop other product columns)
if not all_promos.empty and 'itemcode' in all_promos.columns:
    # Use most recent promo if date exists
    if 'priceupdatedate' in all_promos.columns:
        all_promos['priceupdatedate'] = pd.to_datetime(all_promos['priceupdatedate'], errors='coerce')
        all_promos = all_promos.sort_values('priceupdatedate', ascending=False)
        promos_latest = all_promos.drop_duplicates(subset=['itemcode', 'supermarket'], keep='first')
    else:
        promos_latest = all_promos
    # Only merge on shared columns + keep all product columns
    merge_cols = ['itemcode', 'supermarket']
    promo_cols = [c for c in promos_latest.columns if c not in merge_cols]
    df_final = products_latest.merge(
        promos_latest[merge_cols + promo_cols],
        on=merge_cols,
        how='left',
        suffixes=('', '_promo')
    )
else:
    df_final = products_latest

# Save with all original and promo columns
df_final.to_csv("data/unique_products_with_latest_price_and_promo_FULL.csv", index=False)
print("✅ Saved: data/unique_products_with_latest_price_and_promo_FULL.csv")
print(df_final.head())


✅ Loaded product file for shufersal
✅ Loaded promo file for shufersal
✅ Loaded product file for rami_levy
✅ Loaded promo file for rami_levy
✅ Loaded product file for victory
✅ Loaded promo file for victory
✅ Loaded product file for tiv_taam
✅ Loaded promo file for tiv_taam
✅ Loaded product file for yohananof
✅ Loaded promo file for yohananof
✅ Loaded product file for osher_ad
✅ Loaded promo file for osher_ad
✅ Loaded product file for mega
✅ Loaded promo file for mega
✅ Saved: data/unique_products_with_latest_price_and_promo_FULL.csv
  found_folder file_name  chainid  subchainid  storeid  bikoretno  \
0          NaN       NaN      NaN         NaN      NaN        NaN   
1          NaN       NaN      NaN         NaN      NaN        NaN   
2          NaN       NaN      NaN         NaN      NaN        NaN   
3          NaN       NaN      NaN         NaN      NaN        NaN   
4          NaN       NaN      NaN         NaN      NaN        NaN   

      priceupdatedate      itemcode  itemtype 

In [4]:
EXCLUDE_KEYWORDS = [
    "שמפו", "שמפוו", "סבון", "מרכך", "משחת שיניים", "מברשת שיניים", "נייר טואלט", "חומר ניקוי",
    "טיטולים", "פדים", "תחבושת", "אבקת כביסה", "מרכך כביסה", "כלים חד פעמיים",
    "שקיות אשפה", "שקית אשפה", "נוזל רצפה", "חומרי ניקוי",
    # Add more as needed
    "shampoo", "soap", "detergent", "toothpaste", "toothbrush", "diaper", "pad", "sanitary",
    "cleaner", "dish", "garbage bag", "floor cleaner", "laundry", "disposable"
]


In [6]:
import pandas as pd
import re

def contains_exclude(text):
    text = str(text).lower()
    for kw in EXCLUDE_KEYWORDS:
        if kw in text:
            return True
    return False

# Apply the filter on 'itemname' and optionally 'manufactureitemdescription'
before = len(df_final)
df_final = df_final[~(
    df_final['itemname'].apply(contains_exclude) |
    (df_final['manufactureitemdescription'].apply(contains_exclude) if 'manufactureitemdescription' in df_final.columns else False)
)]
after = len(df_final)
print(f"✅ Filtered out {before-after} non-food/non-cooking products. Remaining: {after}")
df_final.to_csv("data/unique_products_with_latest_price_and_promo_FULL.csv", index=False)
print("✅ Saved: data/unique_products_with_latest_price_and_promo_FULL.csv")
print(df_final.head())

✅ Filtered out 0 non-food/non-cooking products. Remaining: 143736
✅ Saved: data/unique_products_with_latest_price_and_promo_FULL.csv
  found_folder file_name  chainid  subchainid  storeid  bikoretno  \
0          NaN       NaN      NaN         NaN      NaN        NaN   
1          NaN       NaN      NaN         NaN      NaN        NaN   
2          NaN       NaN      NaN         NaN      NaN        NaN   
3          NaN       NaN      NaN         NaN      NaN        NaN   
4          NaN       NaN      NaN         NaN      NaN        NaN   

      priceupdatedate      itemcode  itemtype                    itemname  \
0 2025-06-18 22:32:00  7.290107e+12       NaN        בפלות טורטית 360 גרם   
1 2025-06-18 22:32:00  7.290003e+12       NaN    חמאה צרפתית פלאשרד במשקל   
2 2025-06-18 22:32:00  7.290018e+12       NaN  סוכרזית סוכרלוז 200 טבליות   
3 2025-06-18 22:32:00  7.290001e+12       NaN    סוכרזית קנקן 1200 טבליות   
4 2025-06-18 22:32:00  7.290019e+12       NaN      קמח שקדים כרם - 

In [9]:
COLUMNS_TO_KEEP = [
    'itemcode', 'itemname', 'manufacturername', 'manufacturecountry',
    'manufactureitemdescription', 'unitqty', 'unitofmeasure', 'qtyinpackage',
    'itemprice', 'supermarket', 'itemstatus',
    'promotiondescription', 'discountedprice', 'promotionstartdate', 'promotionenddate'
]
df_final = df_final[[c for c in COLUMNS_TO_KEEP if c in df_final.columns]]


In [10]:
df_final.to_csv("data/unique_products_with_latest_price_and_promo_FULL.csv", index=False)
print("✅ Saved: data/unique_products_with_latest_price_and_promo_FULL.csv")
print(df_final.head())

✅ Saved: data/unique_products_with_latest_price_and_promo_FULL.csv
       itemcode                    itemname manufacturername  \
0  7.290107e+12        בפלות טורטית 360 גרם              NaN   
1  7.290003e+12    חמאה צרפתית פלאשרד במשקל              NaN   
2  7.290018e+12  סוכרזית סוכרלוז 200 טבליות              NaN   
3  7.290001e+12    סוכרזית קנקן 1200 טבליות              NaN   
4  7.290019e+12      קמח שקדים כרם - 250 גר              NaN   

  manufacturecountry  manufactureitemdescription  unitqty unitofmeasure  \
0                NaN                         NaN      NaN           NaN   
1               צרפת                         NaN      NaN           NaN   
2                NaN                         NaN  Unknown       Unknown   
3            לא ידוע                         NaN      NaN           NaN   
4                פרו                         NaN      NaN           NaN   

  qtyinpackage  itemprice supermarket  itemstatus promotiondescription  \
0          NaN       23

In [1]:
!pip install google-cloud-translate pandas


   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   --------- ------------------------------ 1.0/4.3 MB 5.0 MB/s eta 0:00:01
   ------------------------ --------------- 2.6/4.3 MB 5.8 MB/s eta 0:00:01
   -------------------------------------- - 4.2/4.3 MB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 5.9 MB/s eta 0:00:00

   ----------------------------------------  0/14 [pyasn1]
   ----------------------------------------  0/14 [pyasn1]
   ----------------------------------------  0/14 [pyasn1]
   ----------------------------------------  0/14 [pyasn1]
   ----------------------------------------  0/14 [pyasn1]
   -- -------------------------------------  1/14 [protobuf]
   -- -------------------------------------  1/14 [protobuf]
   -- -------------------------------------  1/14 [protobuf]
   -- -------------------------------------  1/14 [protobuf]
   -- -------------------------------------  1/14 [protobuf]
   -- --------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [8]:
import os
import pandas as pd
from google.cloud import translate


# Path to your JSON key file:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "positive-bonbon-463817-h1-492e58264930.json"

# Read your CSV file
df = pd.read_csv('data/unique_products_with_latest_price_and_promo_FULL.csv')  # <-- Change this to your actual filename

# Create the translation client
client = translate.Client()

def batch_translate(texts, src='he', tgt='en'):
    # Google allows up to 128 items per call
    results = client.translate(texts, source_language=src, target_language=tgt)
    return [r['translatedText'] for r in results]

# Batch translation (adjust column name if needed)
col_to_translate = 'itemname'  # <-- Change to your column name
batch_size = 100
translated = []

for i in range(0, len(df), batch_size):
    batch = df[col_to_translate].iloc[i:i+batch_size].astype(str).tolist()
    translated.extend(batch_translate(batch))
    print(f"{i + len(batch)} / {len(df)} translated...")

df['english_col'] = translated

# Save to a new CSV
df.to_csv('israeli_supermarkets.csv', index=False)
print("Done! Translated file saved as israeli_supermarkets.csv")


ModuleNotFoundError: No module named 'google.cloud'

In [6]:
!pip show google-cloud-translate


Name: google-cloud-translate
Version: 3.20.3
Summary: Google Cloud Translate API client library
Home-page: https://github.com/googleapis/google-cloud-python/tree/main/packages/google-cloud-translate
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: C:\Users\Nagham Omar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages
Requires: google-api-core, google-auth, google-cloud-core, grpc-google-iam-v1, proto-plus, protobuf
Required-by: 


In [11]:
import sys
!{sys.executable} -m pip install google-cloud-translate


'C:\Users\Nagham' is not recognized as an internal or external command,
operable program or batch file.
